In [1]:
import json
from pathlib import Path

In [7]:
def load_annot(fname):
    elem_dict = {}
    with open(fname) as f:
        lines = f.readlines()
        for line in lines:
            elem = json.loads(line.strip())
            docId = elem.pop('docId')
            elem_dict[docId] = elem
    return elem_dict
def load_doc(dirname, annot_ids):
    doc_dict = {}
    for annot_id in annot_ids:
        fname = dirname + annot_id + ".json"
        with open(fname) as f:
            doc_elem = json.load(f)
            doc_dict[annot_id] = doc_elem
    return doc_dict

In [5]:
annot_dict = load_annot('../annotation/train.json')
doc_dict = load_doc('/Users/echoi/Project/document-e2e-sent/data/doc/train_data/', annot_dict.keys())

In [20]:
for doc_id, annot_elem in annot_dict.items():
    doc_elem = doc_dict[doc_id]
    print('--')
    print('labels/')
    print([(d['holder'], d['target'], d['sentiment']) for d in annot_elem['sentiments']])
    print('/labels')
    # output would be 'docId':'[{target: ent1, holder:ent2, label: label, context: [[tokens:[], ss: , se: , os: , oe:], [tokens:[], ss: , se: , os: , oe:],
    # [tokens:[], ss: , se: , os: , oe:], [tokens:[], ss: , se: , os: , oe:]]}] 
    print('-- new document --')
    for cluster in doc_elem['cluster_json']:
        print('-cluster-')
        print(cluster)
        print('-/cluster-')
        break
    
    print(len(doc_elem['named_entity'].items()))
    for named_entity, occurences in doc_elem['named_entity'].items():
        print(named_entity, occurences)
        print(doc_elem['headline'])
    for sent in doc_elem['text']:
        print(sent)
    break

--
labels/
[('Reazuddin', 'China', 'Positive'), ('Reazuddin', 'Bangladesh', 'Negative')]
/labels
-- new document --
-cluster-
[{'mention_type': 'PROPER', 'end_ind': 4, 'sent_ind': 1, 'word': 'Bangladesh', 'token_ind': 3}, {'mention_type': 'PROPER', 'end_ind': 10, 'sent_ind': 12, 'word': 'Bangladesh', 'token_ind': 9}, {'mention_type': 'PROPER', 'end_ind': 15, 'sent_ind': 11, 'word': 'Bangladesh', 'token_ind': 14}, {'mention_type': 'NOMINAL', 'end_ind': 15, 'sent_ind': 12, 'word': 'a least-developed country', 'token_ind': 12}, {'mention_type': 'PROPER', 'end_ind': 8, 'sent_ind': 8, 'word': 'Bangladesh', 'token_ind': 7}]
-/cluster-
11
DHAKA [[2, 'LOCATION', 4, 4]]
Interview: Bangladesh should learn from China experience of development: journalist
English [[3, 'MISC', 22, 22]]
Interview: Bangladesh should learn from China experience of development: journalist
Huang Yanan [[2, 'PERSON', 2, 3]]
Interview: Bangladesh should learn from China experience of development: journalist
Bangladeshi [[